In [ ]:
import re
import matplotlib.pyplot as plt

def parse_line(line):
    """
    Parsea una línea con formato:
    [62731,1373430],[62731,1373434],0.000996
    Devuelve: (cost_vector, time)
    Se ignora el primer vector (nodos solución) y se toma el segundo (costos).
    """
    # Extraer los contenidos de los corchetes
    vectors = re.findall(r'\[([^\]]+)\]', line)
    # Extraer el tiempo (último valor luego de la última coma)
    time_match = re.search(r',\s*([\d\.eE+-]+)\s*$', line)
    time_val = float(time_match.group(1)) if time_match else None

    # El segundo vector corresponde a los costos
    if len(vectors) >= 2:
        cost_str = vectors[1]
        cost_values = [float(x) for x in cost_str.split(',')]
    else:
        cost_values = []
    return cost_values, time_val

# Leer el frente exacto del archivo boa.txt (se ignora el tiempo)
exact_costs = []
with open('BOASTART.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        cost_vector, _ = parse_line(line)
        exact_costs.append(cost_vector)

# Leer las soluciones aproximadas de aprox.txt (cada línea incluye el tiempo)
approx_points = []  # Lista de tuplas (tiempo, cost_vector)
with open('AnytimeApexHybrid1.5.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        cost_vector, time_val = parse_line(line)
        approx_points.append((time_val, cost_vector))

# Asegurarse de que los puntos estén ordenados por tiempo
approx_points.sort(key=lambda x: x[0])

# Función para calcular el factor de dominancia entre dos vectores
def dominance_factor(a, p):
    # Calcula DF(a, p) = max( max_i(a_i/p_i - 1), 0 )
    # Se asume que los costos son positivos
    ratios = [(a_i / p_i - 1) for a_i, p_i in zip(a, p)]
    return max(max(ratios), 0)

# Función para calcular el error de aproximación dado un conjunto aproximado A y el frente exacto P*
def approximation_error(approx_set, exact_set):
    # Para cada solución exacta p en P*, se calcula el mínimo DF(a, p) para a en A
    # El error es el máximo de estos valores.
    errors = []
    for p in exact_set:
        min_df = float('inf')
        for a in approx_set:
            df = dominance_factor(a, p)
            if df < min_df:
                min_df = df
        errors.append(min_df)
    return max(errors)

# Calcular el error de aproximación en función del tiempo, acumulando las soluciones aproximadas
cumulative_approxs = []  # Conjunto acumulado de soluciones aproximadas
error_time = []          # Lista de tuplas (tiempo, error)

for t, cost_vec in approx_points:
    cumulative_approxs.append(cost_vec)
    err = approximation_error(cumulative_approxs, exact_costs)
    error_time.append((t, err))

# Extraer listas para la gráfica: tiempos y errores
times = [t for t, err in error_time]
errors = [err for t, err in error_time]

# Graficar Error de Aproximación vs Tiempo
plt.figure(figsize=(10, 6))
plt.plot(times, errors, marker='o', linestyle='-')
plt.xlabel('Tiempo (s)')
plt.ylabel('Error de Aproximación')
plt.title('Disminución del Error de Aproximación vs Tiempo')
plt.grid(True)
plt.show()
